In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from textwrap import dedent
from pprint import pprint

In [3]:
from langchain_community.tools import TavilySearchResults
#from langchain_community.tools import DuckDuckGoSearchResults

query = "스테이크와 어울리는 와인을 추천해 주세요."

web_search = TavilySearchResults(max_results=2)
#web_search = DuckDuckGoSearchResults(max_results=2)

search_results = web_search.invoke(query)

for result in search_results:
    print(result)
    print('-'*100)

H
----------------------------------------------------------------------------------------------------
T
----------------------------------------------------------------------------------------------------
T
----------------------------------------------------------------------------------------------------
P
----------------------------------------------------------------------------------------------------
E
----------------------------------------------------------------------------------------------------
r
----------------------------------------------------------------------------------------------------
r
----------------------------------------------------------------------------------------------------
o
----------------------------------------------------------------------------------------------------
r
----------------------------------------------------------------------------------------------------
(
-----------------------------------------------------------------------

In [4]:
print(type(web_search))

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>


In [5]:
print(web_search.name)

tavily_search_results_json


In [6]:
print(web_search.description)

A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.


In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')

llm_with_tools = llm.bind_tools(tools=[web_search])

In [8]:
query = "안녕하세요."
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print('-'*100)

pprint(ai_msg.content)
print('-'*100)

pprint(ai_msg.tool_calls)
print('-'*100)

AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 82, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'stop', 'logprobs': None}, id='run--f8879844-47f9-43be-b837-fbd716b2b082-0', usage_metadata={'input_tokens': 82, 'output_tokens': 11, 'total_tokens': 93})
----------------------------------------------------------------------------------------------------
'안녕하세요! 어떻게 도와드릴까요?'
----------------------------------------------------------------------------------------------------
[]
----------------------------------------------------------------------------------------------------


In [9]:
query = "스테이크와 어울리는 와인을 추천해 주세요."
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print('-'*100)

pprint(ai_msg.content)
print('-'*100)

pprint(ai_msg.tool_calls)
print('-'*100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_HqVi5LLj1ocHfhY7QRVCGsrf', 'function': {'arguments': '{"query":"스테이크와 어울리는 와인 추천"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 92, 'total_tokens': 119, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--2d5ce4b6-2ee9-4f85-95c9-a73128d1a1d7-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '스테이크와 어울리는 와인 추천'}, 'id': 'call_HqVi5LLj1ocHfhY7QRVCGsrf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 92, 'output_tokens': 27, 'total_tokens': 119})
-------------------------------------------------

In [10]:
tool_call = ai_msg.tool_calls[0]
tool_call

{'name': 'tavily_search_results_json',
 'args': {'query': '스테이크와 어울리는 와인 추천'},
 'id': 'call_HqVi5LLj1ocHfhY7QRVCGsrf',
 'type': 'tool_call'}

In [11]:
tool_message = web_search.invoke(tool_call)

print(tool_message)

content="HTTPError('401 Client Error: Unauthorized for url: https://api.tavily.com/search')" name='tavily_search_results_json' tool_call_id='call_HqVi5LLj1ocHfhY7QRVCGsrf' artifact={}


In [12]:
pprint(tool_message.content)

("HTTPError('401 Client Error: Unauthorized for url: "
 "https://api.tavily.com/search')")


In [13]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain

today = datetime.today().strftime("%Y-%m-%d")

prompt = ChatPromptTemplate([
    ("system", f"You are helpful AI assistant. Today's date is {today}."),
    ('human', "{user_input}"),
    ("placeholder", "{message}"),
])

llm = ChatOpenAI(model='gpt-4o-mini')

llm_with_tools = llm.bind_tools(tools=[web_search])

llm_chain = prompt | llm_with_tools

@chain
def web_search_chain(user_input: str):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_)

    tool_msgs = web_search.batch(ai_msg.tool_calls)
    print("tool_mags: \n", tool_msgs)
    print('-'*100)

    return llm_chain.invoke({**input_, "message": [ai_msg, *tool_msgs]})

response = web_search_chain.invoke("오늘 모엣샹동 샴페인의 원화 가격은 얼마인가요?")

pprint(response.content)




tool_mags: 
 [ToolMessage(content="HTTPError('401 Client Error: Unauthorized for url: https://api.tavily.com/search')", name='tavily_search_results_json', tool_call_id='call_5FIuIg1nQCMJQiWquxepLSK8', artifact={})]
----------------------------------------------------------------------------------------------------
''


In [14]:
# 도구 정의하기
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool
from typing import List

@tool
def search_web(query: str) -> str:
    """Searches the internet for information that does not exist in the database or 
    for the latest information."""

    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)

    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
        ])
       
    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."



In [15]:
query = "스테이크와 어울리는 와인을 추천해 주세요."
search_result = search_web.invoke(query)

print(search_result)

TypeError: string indices must be integers, not 'str'

In [16]:
llm_with_tools = llm.bind_tools(tools=[search_web])
query = "스테이크와 어울리는 와인을 추천해 주세요."
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print('-'*100)

pprint(ai_msg.content)
print('-'*100)

pprint(ai_msg.tool_calls)
print('-'*100)


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_X23Fq4QzC9HTC1kwF7JSf3Ok', 'function': {'arguments': '{"query": "스테이크와 어울리는 와인 추천"}', 'name': 'search_web'}, 'type': 'function'}, {'id': 'call_2FbbUp17nwKhyAPrgYHWePnA', 'function': {'arguments': '{"query": "steak wine pairing recommendations"}', 'name': 'search_web'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 70, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--648c2732-2f66-47c7-8094-88b224032b47-0', tool_calls=[{'name': 'search_web', 'args': {'query': '스테이크와 어울리는 와인 추천'}, 'id': 'call_X23Fq4QzC9HTC1kwF7JSf3Ok', 'type': 'tool_call'},

In [17]:
from langchain.document_loaders import TextLoader

# 메뉴판 텍스트 데이터를 로드
loader = TextLoader("./data/restaurant_menu.txt", encoding="utf-8")
documents = loader.load()

print(len(documents))

1


In [18]:
import re
from langchain_core.documents import Document

# 문서 분할 (Chunking)
def split_menu_items(document):
    """
    메뉴 항목을 분리하는 함수 
    """
    # 정규표현식 정의 
    pattern = r'(\d+.\s.*?)(?=\n\n\d+.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)

# 각 메뉴 항목을 Document 객체로 변환
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        # 메뉴 이름 추출
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()

# 새로운 Document 객체 생성
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)

    return menu_documents


# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

총 10개의 메뉴 항목이 처리되었습니다.

메뉴 번호: 1
메뉴 이름: 시그니처 스테이크
내용:
1. 시그니처 스테이크
   • 가격: ₩35,000
   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스
   • 설명: 셰프의 특제 시그니처 메뉴로, ...

메뉴 번호: 2
메뉴 이름: 트러플 리조또
내용:
2. 트러플 리조또
   • 가격: ₩22,000
   • 주요 식재료: 이탈리아산 아르보리오 쌀, 블랙 트러플, 파르미지아노 레지아노 치즈
   • 설명: 크리미한 텍스처의 리조...


In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document

embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-small", 
)

In [ ]:
# Chroma Vectorstore를 사용하기 위한 준비
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-small", 
)

# Chroma 인덱스 생성
menu_db = Chroma.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model,
    collection_name="restaurant_menu",
    persist_directory="./chroma_db",
)

# Retriever 생성
menu_retriever = menu_db.as_retriever(
    search_kwargs={'k': 2},
)

# 쿼리 테스트
query = "시그니처 스테이크의 가격과 특징은 무엇인가요?"
docs = menu_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


검색 결과: 2개
메뉴 번호: 1
메뉴 이름: 시그니처 스테이크

메뉴 번호: 8
메뉴 이름: 안심 스테이크 샐러드



In [ ]:
# 와인 메뉴 텍스트 데이터를 로드
loader = TextLoader("./data/restaurant_wine.txt", encoding="utf-8")
documents = loader.load()

# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")


# Chroma 인덱스 생성
wine_db = Chroma.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model,
    collection_name="restaurant_wine",
    persist_directory="./chroma_db",
)

wine_retriever = wine_db.as_retriever(
    search_kwargs={'k': 2},
)

query = "스테이크와 어울리는 와인을 추천해주세요."
docs = wine_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


총 10개의 메뉴 항목이 처리되었습니다.

메뉴 번호: 1
메뉴 이름: 샤토 마고 2015
내용:
1. 샤토 마고 2015
   • 가격: ₩450,000
   • 주요 품종: 카베르네 소비뇽, 메를로, 카베르네 프랑, 쁘띠 베르도
   • 설명: 보르도 메독 지역의 프리미엄 ...

메뉴 번호: 2
메뉴 이름: 돔 페리뇽 2012
내용:
2. 돔 페리뇽 2012
   • 가격: ₩380,000
   • 주요 품종: 샤르도네, 피노 누아
   • 설명: 프랑스 샴페인의 대명사로 알려진 프레스티지 큐베입니다. 시트러스...


Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


검색 결과: 2개
메뉴 번호: 10
메뉴 이름: 그랜지 2016

메뉴 번호: 9
메뉴 이름: 샤토 디켐 2015



In [ ]:
# 벡터 저장소 로드
menu_db = Chroma(
    embedding_function=embeddings_model,
    collection_name="restaurant_menu",
    persist_directory="./chroma_db",
)

@tool
def search_menu(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized restaurant menu information from the encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = menu_db.similarity_search(query, k=2)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
from langchain_core.tools import tool
from typing import List
from langchain_core.documents import Document

# 벡터 저장소 로드
wine_db = Chroma(
   embedding_function=embeddings_model,
   collection_name="restaurant_wine",
   persist_directory="./chroma_db",
)

@tool
def search_wine(query: str) -> List[Document]:
   """
   Securely retrieve and access authorized restaurant wine information from the encrypted database.
   Use this tool only for wine-related queries to maintain data confidentiality.
   """
   docs = wine_db.similarity_search(query, k=2)
   if len(docs) > 0:
      return docs

   return [Document(page_content="관련 와인 정보를 찾을 수 없습니다.")]

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
llm_with_tools = llm.bind_tools(tools=[search_menu, search_wine])

query = "시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요."

ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print('-'*100)

pprint(ai_msg.content)
print('-'*100)

pprint(ai_msg.tool_calls)
print('-'*100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wbzK5plGWvdc01yWMTyt9g13', 'function': {'arguments': '{"query": "시그니처 스테이크"}', 'name': 'search_menu'}, 'type': 'function'}, {'id': 'call_u2H83aI9b8rEUVxnszxZyO7r', 'function': {'arguments': '{"query": "스테이크"}', 'name': 'search_wine'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 137, 'total_tokens': 190, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--dcfce7e7-8e55-4ca0-b1f3-7b72eb261e61-0', tool_calls=[{'name': 'search_menu', 'args': {'query': '시그니처 스테이크'}, 'id': 'call_wbzK5plGWvdc01yWMTyt9g13', 'type': 'tool_call'}, {'name': 'search_wine', 'args': {'query

In [ ]:
tools = [search_web, search_wine, search_menu]
for tool in tools:
    print(tool.name)

search_web
search_wine
search_menu


In [ ]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini")

# 3개의 검색 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=tools)

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        if tool_call["name"] == "search_web":
            tool_message = search_web.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "search_wine":
            tool_message = search_wine.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "search_menu":
            tool_message = search_menu.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return llm_chain.invoke({**input, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = restaurant_menu_chain.invoke("시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요.")
#response = restaurant_menu_chain.invoke("전세계에서 가장 비싼 스테이크에 대해 알려 주세요.")

# 응답 출력
print(response.content)

search_menu: 
{'name': 'search_menu', 'args': {'query': '시그니처 스테이크'}, 'id': 'call_BXOEoc8FeJoEl9ZKQuUCBDjW', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
search_wine: 
{'name': 'search_wine', 'args': {'query': '스테이크에 어울리는 와인 추천'}, 'id': 'call_ajvtNfhJc7mGBJ1ZC4vGrKEE', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(metadata={'menu_name': '시그니처 스테이크', 'menu_number': 1, 'source': './data/restaurant_menu.txt'}, page_content='1. 시그니처 스테이크\\n   • 가격: ₩35,000\\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.'), Document(metadata={'menu_name': '안심 스테이크 샐러드', 'menu_number': 8, 'source': './data/restaurant_menu.txt'}, page_content='8. 안심 스테이크 샐러드\\n   • 가격:

In [ ]:
response = restaurant_menu_chain.invoke("파스타 메뉴가 있나요? 이 음식의 역사 또는 유래를 알려주세요.")

pprint(response.content)

search_menu: 
{'name': 'search_menu', 'args': {'query': '파스타'}, 'id': 'call_azqWDVh86iN6ZHl1UzRCoKtr', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(metadata={'menu_name': '해산물 파스타', 'menu_number': 6, 'source': './data/restaurant_menu.txt'}, page_content='6. 해산물 파스타\\n   • 가격: ₩24,000\\n   • 주요 식재료: 링귀네 파스타, 새우, 홍합, 오징어, 토마토 소스\\n   • 설명: 알 덴테로 삶은 링귀네 파스타에 신선한 해산물을 듬뿍 올린 메뉴입니다. 토마토 소스의 산미와 해산물의 감칠맛이 조화를 이루며, 마늘과 올리브 오일로 풍미를 더했습니다. 파슬리를 뿌려 향긋한 맛을 더합니다.'), Document(metadata={'menu_name': '연어 타르타르', 'menu_number': 3, 'source': './data/restaurant_menu.txt'}, page_content='3. 연어 타르타르\\n   • 가격: ₩18,000\\n   • 주요 식재료: 노르웨이산 생연어, 아보카도, 케이퍼, 적양파\\n   • 설명: 신선한 노르웨이산 생연어를 곱게 다져 아보카도, 케이퍼, 적양파와 함께 섞어 만든 타르타르입니다. 레몬 드레싱으로 상큼한 맛을 더했으며, 바삭한 브리오쉬 토스트와 함께 제공됩니다. 전채요리로 완벽한 메뉴입니다.')]", name='search_menu', tool_call_id='call_azqWDVh86iN6ZHl1UzRCoKtr')]
--------------------------------

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate

examples = [
    HumanMessage("트러플 리조또의 가격과 특징, 그리고 어울리는 와인에 대해 알려주세요.", name="example_user"),
    AIMessage("메뉴 정보를 검색하고, 위키피디아에서 추가 정보를 찾은 후, 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "search_menu", "args": {"query": "트러플 리조또"}, "id": "1"}]),
    ToolMessage("트러플 리조또: 가격 ₩28,000, 이탈리아 카나롤리 쌀 사용, 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리", tool_call_id="1"),
    AIMessage("트러플 리조또의 가격은 ₩28,000이며, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다. 이제 추가 정보를 위키피디아에서 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "wiki_summary", "args": {"query": "트러플 리조또", "k": 1}, "id": "2"}]),
    ToolMessage("트러플 리조또는 이탈리아 요리의 대표적인 리조또 요리 중 하나로, 고급 식재료인 트러플을 사용하여 만든 크리미한 쌀 요리입니다. 주로 아르보리오나 카나롤리 등의 쌀을 사용하며, 트러플 오일이나 생 트러플을 넣어 조리합니다. 리조또 특유의 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 것이 특징입니다.", tool_call_id="2"),
    AIMessage("트러플 리조또의 특징에 대해 알아보았습니다. 이제 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "search_wine", "args": {"query": "트러플 리조또에 어울리는 와인"}, "id": "3"}]),
    ToolMessage("트러플 리조또와 잘 어울리는 와인으로는 주로 중간 바디의 화이트 와인이 추천됩니다. 1. 샤르도네: 버터와 오크향이 트러플의 풍미를 보완합니다. 2. 피노 그리지오: 산뜻한 산미가 리조또의 크리미함과 균형을 이룹니다. 3. 베르나차: 이탈리아 토스카나 지방의 화이트 와인으로, 미네랄리티가 트러플과 잘 어울립니다.", tool_call_id="3"),
    AIMessage("트러플 리조또(₩28,000)는 이탈리아의 대표적인 리조또 요리 중 하나로, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다. 주요 특징으로는 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 점입니다. 고급 식재료인 트러플을 사용해 풍부한 맛과 향을 내며, 주로 아르보리오나 카나롤리 등의 쌀을 사용합니다. 트러플 리조또와 잘 어울리는 와인으로는 중간 바디의 화이트 와인이 추천됩니다. 특히 버터와 오크향이 트러플의 풍미를 보완하는 샤르도네, 산뜻한 산미로 리조또의 크리미함과 균형을 이루는 피노 그리지오, 그리고 미네랄리티가 트러플과 잘 어울리는 이탈리아 토스카나 지방의 베르나차 등이 좋은 선택이 될 수 있습니다.", name="example_assistant"),
    ]

system = """You are an AI assistant providing restaurant menu information and general food-related knowledge.
For information about the restaurant's menu, use the search_menu tool.
For wine recommendations or pairing information, use the search_wine tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    examples,  #examples는 리스트의 각 항목을 펼쳐서(unpack) 넣어주는 문법입니다.
    ("human", "{query}"),
])

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 체인 실행
query = "스테이크 메뉴가 있나요? 스테이크와 어울리는 와인을 추천해주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

NotImplementedError: Unsupported message type: <class 'list'>

In [ ]:
# 체인 실행
query = "파스타의 유래에 대해서 알고 있나요? 서울 강남의 파스타 맛집을 추천해주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

NameError: name 'fewshot_search_chain' is not defined

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

agent_prompt = ChatPromptTemplate.from_messages([
    ("system", dedent("""
        You are an AI assistant providing restaurant menu information and general food-related knowledge. 
        Your main goal is to provide accurate information and effective recommendations to users.

        Key guidelines:
        
For restaurant menu information, use the search_menu tool. This tool provides details on menu items, including prices, ingredients, and cooking methods.
For general food information, history, and cultural background, utilize the wiki_summary tool.
For wine recommendations or food and wine pairing information, use the search_wine tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
Provide clear and concise responses based on the search results.
If a question is ambiguous or lacks necessary information, politely ask for clarification.
Always maintain a helpful and professional tone.
When providing menu information, describe in the order of price, main ingredients, and distinctive cooking methods.
When making recommendations, briefly explain the reasons.
Maintain a conversational, chatbot-like style in your final responses. Be friendly, engaging, and natural in your communication.


        Remember, understand the purpose of each tool accurately and use them in appropriate situations. 
        Combine the tools to provide the most comprehensive and accurate answers to user queries. 
        Always strive to provide the most current and accurate information.
        """)),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
    # 에이전트가 도구를 호출한 흔적, 즉 생각 과정 + 도구 호출 요청 + 도구 응답 등을 여기에 넣습니다.
])

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

tools = [search_web, search_menu, search_wine]
agent = create_tool_calling_agent(llm, tools, agent_prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
query = "시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요."

agent_response = agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...

Invoking: `search_menu` with `{'query': '시그니처 스테이크'}`


[Document(metadata={'menu_name': '시그니처 스테이크', 'menu_number': 1, 'source': './data/restaurant_menu.txt'}, page_content='1. 시그니처 스테이크\n   • 가격: ₩35,000\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.'), Document(metadata={'menu_name': '안심 스테이크 샐러드', 'menu_number': 8, 'source': './data/restaurant_menu.txt'}, page_content='8. 안심 스테이크 샐러드\n   • 가격: ₩26,000\n   • 주요 식재료: 소고기 안심, 루꼴라, 체리 토마토, 발사믹 글레이즈\n   • 설명: 부드러운 안심 스테이크를 얇게 슬라이스하여 신선한 루꼴라 위에 올린 메인 요리 샐러드입니다. 체리 토마토와 파마산 치즈 플레이크로 풍미를 더하고, 발사믹 글레이즈로 마무리하여 고기의 풍미를 한층 끌어올렸습니다.')]
Invoking: `search_wine` with `{'query': '스테이크'}`


[Document(metadata={'menu_name': '샤토 디켐 2015', 'menu_number': 9, 'source': './data/restaurant_wine.txt'}, page_content='9. 샤토 디켐 2015\n   • 가격: ₩800,000 (375ml)\n   • 주요 품종: 세미